In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [3]:
# Load data
train_data=pd.read_csv("/kaggle/input/playground-series-s5e4/train.csv")
test_data=pd.read_csv("/kaggle/input/playground-series-s5e4/test.csv")

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           750000 non-null  int64  
 1   Podcast_Name                 750000 non-null  object 
 2   Episode_Title                750000 non-null  object 
 3   Episode_Length_minutes       662907 non-null  float64
 4   Genre                        750000 non-null  object 
 5   Host_Popularity_percentage   750000 non-null  float64
 6   Publication_Day              750000 non-null  object 
 7   Publication_Time             750000 non-null  object 
 8   Guest_Popularity_percentage  603970 non-null  float64
 9   Number_of_Ads                749999 non-null  float64
 10  Episode_Sentiment            750000 non-null  object 
 11  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 68.7+ MB


In [5]:
train_data.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


In [6]:
# Remove unnecessary columns 
train_data.drop(["id","Podcast_Name","Episode_Title"], axis=1, inplace=True)
test_data.drop(["Podcast_Name","Episode_Title"], axis=1, inplace=True)

In [7]:
# Missing‑value percentage 
train_data.isna().sum()/len(train_data)*100

Episode_Length_minutes         11.612400
Genre                           0.000000
Host_Popularity_percentage      0.000000
Publication_Day                 0.000000
Publication_Time                0.000000
Guest_Popularity_percentage    19.470667
Number_of_Ads                   0.000133
Episode_Sentiment               0.000000
Listening_Time_minutes          0.000000
dtype: float64

In [8]:
test_data.isna().sum()/len(test_data)*100

id                              0.0000
Episode_Length_minutes         11.4944
Genre                           0.0000
Host_Popularity_percentage      0.0000
Publication_Day                 0.0000
Publication_Time                0.0000
Guest_Popularity_percentage    19.5328
Number_of_Ads                   0.0000
Episode_Sentiment               0.0000
dtype: float64

In [9]:
# Number_of_Ads rows missing in train are removed
train_data.dropna(subset=["Number_of_Ads"], inplace=True)

In [10]:
# Summary stats
train_data.describe()

,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,662906.000000,749999.000000,603969.000000,749999.000000,749999.000000
mean,64.504782,59.859944,52.236453,1.348855,45.437435
std,32.969609,22.873082,28.451265,1.151130,27.138313
min,0.000000,1.300000,0.000000,0.000000,0.000000
25%,35.730000,39.410000,28.380000,0.000000,23.178350
50%,63.840000,60.050000,53.580000,1.000000,43.379460
75%,94.070000,79.530000,76.600000,2.000000,64.811580
max,325.240000,119.460000,119.910000,103.910000,119.970000


In [ ]:
# Visual outlier inspection
import plotly.express as px
px.box(train_data, x="Episode_Length_minutes").show()

In [12]:
px.box(train_data, x="Number_of_Ads").show()

In [13]:
# Impute remaining numerical NaNs

# Episode_Length_minutes - median (robust to outliers)
train_data["Episode_Length_minutes"]=train_data["Episode_Length_minutes"].fillna(train_data["Episode_Length_minutes"].median())
test_data["Episode_Length_minutes"]=test_data["Episode_Length_minutes"].fillna(test_data["Episode_Length_minutes"].median())

In [14]:
# Guest_Popularity_percentage - mean
train_data["Guest_Popularity_percentage"]=train_data["Guest_Popularity_percentage"].fillna(train_data["Guest_Popularity_percentage"].mean())
test_data["Guest_Popularity_percentage"]=test_data["Guest_Popularity_percentage"].fillna(test_data["Guest_Popularity_percentage"].mean())

In [15]:
# Helper to flag IQR outliers
def find_outliers_IQR(col):
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    IQR = q3 - q1
    outliers = col[(col<(q1-1.5*IQR)) | (col>(q3+1.5*IQR))]
    return outliers

In [16]:
# Find indexes of outliers
find_outliers_IQR(train_data["Episode_Length_minutes"])

101637    325.24
Name: Episode_Length_minutes, dtype: float64

In [17]:
# Drop extreme episode‑length outlier manually spotted
train_data.drop(train_data.index[[101637]], inplace=True)

In [18]:
ads_outliers = find_outliers_IQR(train_data["Number_of_Ads"])

In [19]:
# Drop outlier rows by index label
train_data.drop(index=ads_outliers.index, inplace=True)

In [20]:
# One‑hot‑encode the categorical columns 
train_data=pd.get_dummies(train_data,columns=["Genre", "Publication_Day", "Publication_Time", "Episode_Sentiment"],drop_first=True)
test_data=pd.get_dummies(test_data,columns=["Genre", "Publication_Day", "Publication_Time", "Episode_Sentiment"],drop_first=True)

In [21]:
# Split features and target column for model training
X=train_data.drop("Listening_Time_minutes", axis=1)
y=train_data["Listening_Time_minutes"]

In [22]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 749989 entries, 0 to 749999
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Episode_Length_minutes       749989 non-null  float64
 1   Host_Popularity_percentage   749989 non-null  float64
 2   Guest_Popularity_percentage  749989 non-null  float64
 3   Number_of_Ads                749989 non-null  float64
 4   Genre_Comedy                 749989 non-null  bool   
 5   Genre_Education              749989 non-null  bool   
 6   Genre_Health                 749989 non-null  bool   
 7   Genre_Lifestyle              749989 non-null  bool   
 8   Genre_Music                  749989 non-null  bool   
 9   Genre_News                   749989 non-null  bool   
 10  Genre_Sports                 749989 non-null  bool   
 11  Genre_Technology             749989 non-null  bool   
 12  Genre_True Crime             749989 non-null  bool   
 13  Publ

In [23]:
from sklearn.preprocessing import StandardScaler

num_cols = ["Episode_Length_minutes","Host_Popularity_percentage","Guest_Popularity_percentage","Number_of_Ads"]

# keep only the numeric predictors
train_numeric = X[num_cols]          
test_numeric  = test_data[num_cols]

scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_numeric)  # fit on train, scale train
test_scaled  = scaler.transform(test_numeric)       # scale test with same params

# rebuild DataFrames with the scaled numeric columns
X[num_cols] = train_scaled
test_data[num_cols] = test_scaled

In [24]:
X.head()

,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Genre_Comedy,Genre_Education,Genre_Health,Genre_Lifestyle,Genre_Music,Genre_News,...,Publication_Day_Saturday,Publication_Day_Sunday,Publication_Day_Thursday,Publication_Day_Tuesday,Publication_Day_Wednesday,Publication_Time_Evening,Publication_Time_Morning,Publication_Time_Night,Episode_Sentiment_Neutral,Episode_Sentiment_Positive
0,-0.018936,0.653600,-0.000006,-1.213256,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,True
1,1.786495,0.309964,0.928786,0.586979,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,0.305629,0.441997,-1.694630,-1.213256,False,True,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False
3,0.088500,-0.115426,1.036495,0.586979,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
4,1.486773,0.883564,0.252369,1.487096,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [25]:
from sklearn.model_selection import train_test_split 
#split dataset into train and test data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=14, stratify=y)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
import numpy as np

error = []

# Calculating error for K values between 1 and 40
for i in range(1, 40):
    knn = KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    error.append(np.mean((pred_i - y_test) ** 2))  # mean squared error

plt.figure(figsize=(12, 6))
plt.plot(range(1, 40), error, color='red', linestyle='dashed', marker='o', markerfacecolor='blue', markersize=10)
plt.title('MSE vs. K (KNN Regressor)')
plt.xlabel('K Value')
plt.ylabel('Mean Squared Error')
plt.savefig('knn_mse_vs_k.png')
plt.show()

In [ ]:
best_k = np.argmin(error) + 1
print(f"Best k = {best_k},  MSE = {error[best_k-1]:.4f}")

In [ ]:
# Re-train the best k and predict
knn_best = KNeighborsRegressor(n_neighbors=best_k)
knn_best.fit(X_train, y_train)
y_pred = knn_best.predict(X_test)

In [ ]:
# Evaluate the model using R-squared
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.4f}')
print(f'R-squared: {r2:.4f}')

In [ ]:
# Improving the kNN model’s performance with Hyperparameter Tuning

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Create a new kNN model
knn = KNeighborsRegressor()

# Create a dictionary of all values we want to test for n_neighbors and distance metrics
param_grid = {'n_neighbors': np.arange(1, 25), 'metric': ['euclidean', 'manhattan']}

# Use GridSearchCV to test all values for n_neighbors and distance metrics
knn_gscv = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model to data
knn_gscv.fit(X_train, y_train)

In [ ]:
# Check best performing n_neighbors’ value 
print(f"Best Parameters: {knn_gscv.best_params_}")
print(f"Best Score (Negative MSE): {knn_gscv.best_score_}")

In [ ]:
# Perform testing on test dataset 
y_pred = knn_gscv.predict(X_test)

# Calculate the Mean Squared Error and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared: {r2:.4f}")

In [ ]:
# Perform testing on real test data
predictions = knn_gscv.predict(test_data)
output = pd.DataFrame({"id": test_data.id, "Listening_Time_minutes": predictions})

# Save the predictions to a CSV file
output.to_csv("submission_knn.csv", index=False)